In [1]:
#!pip -q install langchain openai tiktoken

In [2]:
import os
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")

In [3]:
from langchain.llms import OpenAI
from langchain.callbacks import get_openai_callback

## Wikipedia

In [4]:
#!pip -q install wikipedia

In [5]:
from langchain.utilities import WikipediaAPIWrapper

In [6]:
wikipedia = WikipediaAPIWrapper()

In [7]:
#wikipedia.run('Langchain')

## Python REPL

In [8]:
#from langchain.utilities import PythonREPL

In [9]:
#python_repl = PythonREPL()

In [10]:
#python_repl.run("print(17*2)")("print(17*2)")

## DuckDuckGo

In [11]:
!pip -q install duckduckgo-search

In [12]:
#query = "who whon the 1990 world cup"

In [13]:
from itertools import islice
from duckduckgo_search import DDGS

class DDG(DDGS):
    def run (query:str):
        '''
        "snippet": result["body"],
        "title": result["title"],
        "link": result["href"],
        '''
        results = DDGS().text(query)
        generator = [result['body'] for result in results]

        # Get the first three results from the generator
        first_three_results = list(islice(generator, 3))
        return first_three_results

In [14]:
# from langchain import OpenAI
# llm = OpenAI(temperature=0)

# This is way cheaper! 
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613", max_tokens=1000)

In [15]:
from io import StringIO
import sys
from typing import Dict, Optional

class PythonREPL:
    """Simulates a standalone Python REPL."""

    def __init__(self):
        pass        

    def run(self, command: str) -> str:
        """Run command and returns anything printed."""
        # sys.stderr.write(command)
        old_stdout = sys.stdout
        sys.stdout = mystdout = StringIO()
        try:
            exec(command, globals())
            sys.stdout = old_stdout
            output = mystdout.getvalue()
        except Exception as e:
            sys.stdout = old_stdout
            output = str(e)
        # sys.stderr.write("PYTHON OUTPUT: \"" + output + "\"\n")
        return output

## Math chain

In [17]:
from langchain.agents import Tool
from langchain import LLMMathChain

#llm_math_chain = LLMMathChain(llm=llm, verbose=True) <- Deprecated !!!
llm_math_chain = LLMMathChain.from_llm(llm=llm, verbose=True)
math_tool = Tool.from_function(
        func=llm_math_chain.run,
        name="Calculator",
        description="Useful for when you need to answer questions about math. This tool is only for math questions and nothing else. Only input math expressions.",
    )

In [18]:


tools = [
    Tool(
        name = "Python REPL",
        func=PythonREPL().run,
        description="useful for when you need to use python to answer a question. You should input python code"
    ),
    Tool(
        name='wikipedia',
        func= wikipedia.run,
        description="Useful for when you need to look up a topic, country or person on wikipedia"
    ),
    Tool(
        name='DuckDuckGo Search',
        func= DDG.run,
        description="Useful for when you need to do a search on the internet to find information that another tool can't find. be specific with your input."
)]

In [19]:
tools.append(math_tool) # example of adding a tool retrospectively:

In [20]:
from langchain.agents import initialize_agent

zero_shot_agent = initialize_agent(
    agent="zero-shot-react-description",
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=3,
)

In [21]:
query_1 = "calculate time to drive 100 miles at 70 miles per hour, convert your answer into hours and minutes"

In [22]:
with get_openai_callback() as cb:
    result = zero_shot_agent.run(query_1)
    print(f"Total Tokens: {cb.total_tokens}")
    print(f"Prompt Tokens: {cb.prompt_tokens}")
    print(f"Completion Tokens: {cb.completion_tokens}")
    print(f"Total Cost (USD): ${cb.total_cost}")
    
result



> Entering new  chain...
I need to calculate the time it takes to drive 100 miles at a speed of 70 miles per hour. Then, I need to convert the answer into hours and minutes.
Action: Calculator
Action Input: 100 / 70

> Entering new  chain...
100 / 70```text
100 / 70
```
...numexpr.evaluate("100 / 70")...

Answer: 1.4285714285714286
> Finished chain.

Observation: Answer: 1.4285714285714286
Thought:I have the time it takes to drive 100 miles at a speed of 70 miles per hour. Now I need to convert it into hours and minutes.
Action: Python REPL
Action Input: 1.4285714285714286 * 60
Observation: 
Thought:I now know the final answer
Final Answer: It takes approximately 1 hour and 25 minutes to drive 100 miles at a speed of 70 miles per hour.

> Finished chain.
Total Tokens: 1340
Prompt Tokens: 1184
Completion Tokens: 156
Total Cost (USD): $0.002088


'It takes approximately 1 hour and 25 minutes to drive 100 miles at a speed of 70 miles per hour.'

# Python REPL

In [28]:
from langchain.agents.agent_toolkits import create_python_agent
from langchain.tools.python.tool import PythonREPLTool
from langchain.python import PythonREPL
from langchain.llms.openai import OpenAI
from langchain.agents.agent_types import AgentType
from langchain.chat_models import ChatOpenAI

agent_executor = create_python_agent(
    llm=ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613"),
    tool=PythonREPLTool(),
    verbose=True,
    agent_type=AgentType.OPENAI_FUNCTIONS,
    agent_executor_kwargs={"handle_parsing_errors": True},
)

In [29]:
question_04 = "what is the 9th prime number?"

In [30]:
with get_openai_callback() as cb:
    response = agent_executor.run({"input": question_04})
    print(f"Total Tokens: {cb.total_tokens}")
    print(f"Prompt Tokens: {cb.prompt_tokens}")
    print(f"Completion Tokens: {cb.completion_tokens}")
    print(f"Total Cost (USD): ${cb.total_cost}")



> Entering new  chain...
Could not parse tool input: {'name': 'python', 'arguments': 'def is_prime(n):\n    if n <= 1:\n        return False\n    for i in range(2, int(n**0.5) + 1):\n        if n % i == 0:\n            return False\n    return True\n\ncount = 0\nnum = 2\nwhile count < 9:\n    if is_prime(num):\n        count += 1\n    num += 1\n\nnum'} because the `arguments` is not valid JSON.Invalid or incomplete response
Invoking: `Python_REPL` with `def is_prime(n):
    if n <= 1:
        return False
    for i in range(2, int(n**0.5) + 1):
        if n % i == 0:
            return False
    return True

count = 0
num = 2
while count < 9:
    if is_prime(num):
        count += 1
    num += 1

num`


The 9th prime number is 23.

> Finished chain.
Total Tokens: 1186
Prompt Tokens: 970
Completion Tokens: 216
Total Cost (USD): $0.0018869999999999998


In [31]:
query_1="if I  will be half my dad's age next year and he is going to be 60 how old am I now?"

In [32]:
with get_openai_callback() as cb:
    response = zero_shot_agent.run(query_1)
    print(f"Total Tokens: {cb.total_tokens}")
    print(f"Prompt Tokens: {cb.prompt_tokens}")
    print(f"Completion Tokens: {cb.completion_tokens}")
    print(f"Total Cost (USD): ${cb.total_cost}")



> Entering new  chain...
I need to calculate my current age based on the information given.
Action: Calculator
Action Input: 60 - (1 + 0.5)

> Entering new  chain...
60 - (1 + 0.5)```text
60 - (1 + 0.5)
```
...numexpr.evaluate("60 - (1 + 0.5)")...

Answer: 58.5
> Finished chain.

Observation: Answer: 58.5
Thought:I now know the final answer
Final Answer: I am currently 58.5 years old.

> Finished chain.
Total Tokens: 869
Prompt Tokens: 787
Completion Tokens: 82
Total Cost (USD): $0.0013445


# PALChain 🦜

In [47]:
from langchain.chains import PALChain
from langchain.prompts import PromptTemplate
from langchain.callbacks import get_openai_callback
from langchain.llms import OpenAI

# This is way cheaper! 
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613", max_tokens=1000)

# create chain
pal_chain = PALChain.from_math_prompt(llm, verbose=True)

# the question we want answered
question_02= "If I drive at 60 miles per hour except for"\
    "the first 20 minutes of the journey from my house"\
    "when I have to do 30 miles per hour,"\
    "calculate how long will it take me to drive to London"\
    "and return back home - London is 100 miles away"\
    "I stop for 15 minutes for petrol and a piss on the way back?"\
    "please answer in hours and minutes!"

# run the chain + check cost
with get_openai_callback() as cb:
    result = pal_chain({"question": question_02})
    print(f"Total Tokens: {cb.total_tokens}")
    print(f"Prompt Tokens: {cb.prompt_tokens}")
    print(f"Completion Tokens: {cb.completion_tokens}")
    print(f"Total Cost (USD): ${cb.total_cost}")

# print the answer
res = result
res.get('result')
print(f"\nFinal Result = {res['result']}")



> Entering new  chain...
def solution():
    """If I drive at 60 miles per hour except for the first 20 minutes of the journey from my house when I have to do 30 miles per hour, calculate how long will it take me to drive to London and return back home - London is 100 miles away. I stop for 15 minutes for petrol and a pit stop on the way back. Please answer in hours and minutes!"""
    distance = 100  # miles
    speed_normal = 60  # miles per hour
    speed_slow = 30  # miles per hour
    time_normal = distance / speed_normal  # hours
    time_slow = 20 / 60  # hours
    time_total = time_normal + time_slow + time_normal  # hours
    time_total_minutes = time_total * 60  # minutes
    time_total_minutes += 15  # add 15 minutes for the pit stop
    hours = int(time_total_minutes // 60)
    minutes = int(time_total_minutes % 60)
    result = (hours, minutes)
    return result

> Finished chain.
Total Tokens: 1400
Prompt Tokens: 1175
Completion Tokens: 225
Total Cost (USD): $0.0022125
